In [1]:
from utilsADCN import mnistLoader, plotPerformance
from ADCNbasic import ADCN
from ADCNmainloop import ADCNmainTrainTest
import numpy as np
import pdb
import torch
import random
from torchvision import datasets, transforms

In [2]:
# random seed control
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

In [3]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# load the training and test datasets
labeledData   = datasets.MNIST(root='data', train=False,download=True, transform=transform)
unlabeledData = datasets.MNIST(root='data', train=True, download=True, transform=transform)

In [4]:
dataStream = mnistLoader(labeledData, unlabeledData)
originalData = dataStream.unlabeledData.clone()
originalLabel= dataStream.unlabeledLabel.clone()

Number of output:  10
Number of labeled data:  10000
Number of unlabeled data:  60000
Number of unlabeled data batch:  60


In [5]:
device = torch.device('cuda:1')

In [6]:
errors = []

In [7]:
for iFold in range(0,10):
    print('Fold :',iFold)
    dataStream.labeledData  = originalData[(5000+(iFold-1)*5000):(10000+(iFold-1)*5000)]
    dataStream.labeledLabel = originalLabel[(5000+(iFold-1)*5000):(10000+(iFold-1)*5000)]

    # preparing testing data   
    dataStream.unlabeledData  = torch.cat((originalData [0:(5000+(iFold-1)*5000)],originalData [(10000+(iFold-1)*5000):50000]),0)
    dataStream.unlabeledLabel = torch.cat((originalLabel[0:(5000+(iFold-1)*5000)],originalLabel[(10000+(iFold-1)*5000):50000]),0)
    
    # random seed control
    np.random.seed(0)
    torch.manual_seed(0)
    random.seed(0)
    
    # training testing
    ADCNnet = ADCN(dataStream.nOutput)
    ADCNnet.evolving = False
    accuracy = ADCNmainTrainTest(ADCNnet, dataStream, trainingBatchSize = 32, device = device)
    errors.append(100 - accuracy)
    print('Error: ', np.round(100 - accuracy,2))

Fold : 0
Network initialization phase is started


Cluster initialization phase is started
A cluster was created containing  2453  centroids
Error:  26.41
Fold : 1
Network initialization phase is started


Cluster initialization phase is started
A cluster was created containing  2354  centroids
Error:  26.56
Fold : 2
Network initialization phase is started


Cluster initialization phase is started
A cluster was created containing  2370  centroids
Error:  26.0
Fold : 3
Network initialization phase is started


Cluster initialization phase is started
A cluster was created containing  2451  centroids
Error:  25.45
Fold : 4
Network initialization phase is started


Cluster initialization phase is started
A cluster was created containing  2449  centroids
Error:  26.79
Fold : 5
Network initialization phase is started


Cluster initialization phase is started
A cluster was created containing  2453  centroids
Error:  25.74
Fold : 6
Network initialization phase is started


Cluster initializati

In [8]:
errors

[26.41111111111111,
 26.557777777777773,
 26.004444444444445,
 25.451111111111118,
 26.78888888888889,
 25.73777777777778,
 25.577777777777783,
 28.422222222222217,
 27.948888888888888,
 24.206666666666663]

In [9]:
print('Accuracy: ', np.round(np.max(errors),2),'+/-',np.round(np.std(errors)**2,2))

Accuracy:  28.42 +/- 1.36
